In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

from numpy.polynomial.polynomial import polyfit

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.api as sm
#import statsmodels
#import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


#### "Is the retaliation directed towards counties that vote for President Trump?" 

Since the posting of the [paper](https://www.nber.org/papers/w26353), I have been asked this question many times:

"Is the retaliation directed towards counties that vote for President Trump?"

Eyeballing the map in Figure 1 of the paper suggest this is the case. Below I followed up with some basic regressions to explore this. Short answer is yes, but most the variation is about places that export the most to china. The interesting observation is that there is a strong correlation between those that exported a lot to china and the propensity to vote for President Trump.

**Note I** This notebook is rough and prelimnary. Treat it as such. 

**Note II** The political economy aspect of the trade war is not what I'm interested. These results might end up in some footnote, but they are not a key part of the paper. Plus they are consistent with the [Fajgelbaum, Goldberg, Kennedy, and Khandelwal (2019)](https://www.nber.org/papers/w25638)


In [2]:
file_path = os.getcwd()

url = "https://github.com/mwaugh0328/consumption_and_tradewar/raw/master/data/countypres_2000-2016.csv"

df = pd.read_csv(url, dtype = {"FIPS": str})

# The data are from here: https://electionlab.mit.edu/data I can't figure out how to create a direct link, so 
# I will just post the data in the respository. 

df = df[df.year == 2016]
# Just look at Trump election (?) does it correlate with other republican canidates?

df["area_fips"] = df.FIPS.astype(str)

df = df[~df.FIPS.isna()]
# Some places with no fips (look like write in counts)

In [3]:
df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,area_fips
41050,2016,Alabama,AL,Autauga,1001,President,Hillary Clinton,democrat,5936.0,24973,20190722,1001
41051,2016,Alabama,AL,Autauga,1001,President,Donald Trump,republican,18172.0,24973,20190722,1001
41052,2016,Alabama,AL,Autauga,1001,President,Other,NaN,865.0,24973,20190722,1001
41053,2016,Alabama,AL,Baldwin,1003,President,Hillary Clinton,democrat,18458.0,95215,20190722,1003
41054,2016,Alabama,AL,Baldwin,1003,President,Donald Trump,republican,72883.0,95215,20190722,1003


In [4]:
def add_zero(x):
    if len(x) == 4:
        x = '0' + x
    else:
        x = x 
    return x

In [5]:
df["area_fips"] = df.area_fips.apply(add_zero)

In [6]:
df["vote_share"] = df.candidatevotes / df.totalvotes

In [7]:
df_trump = df[df.candidate == "Donald Trump"]
# Just look at the trump share

In [8]:
df_trump

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,area_fips,vote_share
41051,2016,Alabama,AL,Autauga,1001,President,Donald Trump,republican,18172.0,24973,20190722,01001,0.727666
41054,2016,Alabama,AL,Baldwin,1003,President,Donald Trump,republican,72883.0,95215,20190722,01003,0.765457
41057,2016,Alabama,AL,Barbour,1005,President,Donald Trump,republican,5454.0,10469,20190722,01005,0.520967
41060,2016,Alabama,AL,Bibb,1007,President,Donald Trump,republican,6738.0,8819,20190722,01007,0.764032
41063,2016,Alabama,AL,Blount,1009,President,Donald Trump,republican,22859.0,25588,20190722,01009,0.893348
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50498,2016,Wyoming,WY,Sweetwater,56037,President,Donald Trump,republican,12154.0,17130,20190722,56037,0.709515
50501,2016,Wyoming,WY,Teton,56039,President,Donald Trump,republican,3921.0,12627,20190722,56039,0.310525
50504,2016,Wyoming,WY,Uinta,56041,President,Donald Trump,republican,6154.0,8470,20190722,56041,0.726564
50507,2016,Wyoming,WY,Washakie,56043,President,Donald Trump,republican,2911.0,3814,20190722,56043,0.763241


This stuff below basically mimics what goes on in the mapping notebook. Grab the trade data, will look at the cross-section as of December 2018. 

In [9]:
file_path = os.getcwd()

trade_data = pq.read_table(file_path + "\\data\\total_trade_data.parquet").to_pandas()

trade_data["time"] = pd.to_datetime(trade_data.time)

trade_data.set_index(["area_fips", "time"],inplace = True)

trade_data["tariff_change"] = trade_data.groupby(["area_fips"]).tariff.diff(12)

trade_data.sort_values(["area_fips", "time"], inplace = True)

In [10]:
tariff_df = trade_data.xs('2018-12-1', level=1).copy()

idx = pd.IndexSlice

tariff_df["china_trade_2017"] = trade_data.loc[idx[:,"2017"],:].groupby(["area_fips"]).china_exp_pc.sum()
# This will create an annual trade flow measure by summing up exports per capita over the all months
# in 2017 (before the war started)

tariff_df["fips_code"] = tariff_df.index

tariff_df["fips_code"] = tariff_df["fips_code"].astype(int)

In [11]:
tariff_df.head()

,total_exp_pc,china_exp_pc,tariff,emplvl_2017,fips,total_employment,tariff_change,china_trade_2017,fips_code
area_fips,,,,,,,,,
10001,524.630933,23.745690,2.426361,2843.0,10001,29514.0,1.357466,565.842217,10001
10003,499.745029,39.971812,0.456547,9072.0,10003,249775.0,0.245212,442.806468,10003
10005,920.169247,52.514965,3.579051,9358.0,10005,60389.0,1.942987,918.184061,10005
1001,2032.472813,220.403712,2.185990,891.0,1001,6100.0,0.844549,2594.575079,1001
1003,728.597752,55.161192,1.358526,4993.0,1003,58645.0,0.822081,953.941297,1003


In [12]:
tariff_trump = tariff_df.merge(df_trump[["vote_share", "FIPS"]], left_on = "area_fips", right_on = "FIPS",
                              indicator = True, how = "left")

In [13]:
tariff_trump["const"] = 1

tariff_trump["log_exp_china"] = np.log(tariff_trump["china_trade_2017"]).replace(-np.inf, np.nan)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Here's how I will pose the question. Ok, what could China be doing? Obvious answer is, place the tariff the stuff we export the most from. We buy a lot of soybeans and pork, well no more...so put the tariff there. This is what this regression does below. Correlate the tariff change with exports in 2017...

In [14]:
all_vars = ["const", "tariff_change", 'vote_share', "log_exp_china", "total_employment"]

dataset = tariff_trump[all_vars].dropna()

exog_vars = ["const", "log_exp_china"]

mod = sm.WLS(dataset.tariff_change, dataset[exog_vars], weights = dataset.total_employment)

res = mod.fit(cov_type='HC3')

print(res.summary())

                            WLS Regression Results                            
Dep. Variable:          tariff_change   R-squared:                       0.378
Model:                            WLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     113.8
Date:                Fri, 11 Oct 2019   Prob (F-statistic):           4.43e-26
Time:                        07:33:53   Log-Likelihood:                -5169.8
No. Observations:                2850   AIC:                         1.034e+04
Df Residuals:                    2848   BIC:                         1.036e+04
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -2.1930      0.291     -7.526

Ok: The more a county was exposed to chinese exports, the harder it was hit. Second, this explains like 38 percent of the variation in change in the tariff.

Now lets add President Trump's share of the vote...

In [15]:
all_vars = ["const", "tariff_change", 'vote_share', "log_exp_china", "total_employment"]

dataset = tariff_trump[all_vars].dropna()

exog_vars = ["const", 'vote_share', "log_exp_china"]

mod = sm.WLS(dataset.tariff_change, dataset[exog_vars], weights = dataset.total_employment)

res = mod.fit(cov_type='HC3')

print(res.summary())

                            WLS Regression Results                            
Dep. Variable:          tariff_change   R-squared:                       0.407
Model:                            WLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     41.01
Date:                Fri, 11 Oct 2019   Prob (F-statistic):           2.75e-18
Time:                        07:33:53   Log-Likelihood:                -5103.3
No. Observations:                2850   AIC:                         1.021e+04
Df Residuals:                    2847   BIC:                         1.023e+04
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -2.2303      0.353     -6.321

What do we see: Export coefficient did not change much. The vote share coefficient is positive meaning: places that had a larger share of the vote going towards President Trump, have a larger increase in their tariff. The other issue to notice as well is that explanatory power only increases by about 2-3 percent. Here is just the vote share.

In [16]:
all_vars = ["const", "tariff_change", 'vote_share', "log_exp_china", "total_employment"]

dataset = tariff_trump[all_vars].dropna()

exog_vars = ["const", 'vote_share']

mod = sm.WLS(dataset.tariff_change, dataset[exog_vars], weights = dataset.total_employment)

res = mod.fit(cov_type='HC3')

print(res.summary())

                            WLS Regression Results                            
Dep. Variable:          tariff_change   R-squared:                       0.132
Model:                            WLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     137.8
Date:                Fri, 11 Oct 2019   Prob (F-statistic):           4.22e-31
Time:                        07:33:53   Log-Likelihood:                -5645.1
No. Observations:                2850   AIC:                         1.129e+04
Df Residuals:                    2848   BIC:                         1.131e+04
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1957      0.067      2.929      0.0

The coefficient is bigger, explanatory power looks larger. **Issue is that there is a correlation between places exporting a lot to china and the vote share for President Trump** This is the real puzzle (as many have pointed out) that it appears places are voting in direct conflict with their economic interest. The regression below confirms this observation.

In [17]:
all_vars = ["const", "tariff_change", 'vote_share', "log_exp_china", "total_employment"]

dataset = tariff_trump[all_vars].dropna()

exog_vars = ["const",  "log_exp_china"]

mod = sm.WLS(dataset.vote_share, dataset[exog_vars], weights = dataset.total_employment)

res = mod.fit(cov_type='HC3')

print(res.summary())

                            WLS Regression Results                            
Dep. Variable:             vote_share   R-squared:                       0.111
Model:                            WLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     17.79
Date:                Fri, 11 Oct 2019   Prob (F-statistic):           2.54e-05
Time:                        07:33:53   Log-Likelihood:                -1421.5
No. Observations:                2850   AIC:                             2847.
Df Residuals:                    2848   BIC:                             2859.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0468      0.091      0.512